In [5]:
import polars as pl
from src.tank_ts_endpoint import fetch_tank_ts_data, GetTanksTimeDataReq

req = GetTanksTimeDataReq(scada_id='98840W2')

df = await fetch_tank_ts_data(req)
if df is None:
    raise ValueError("df is None")
lf = df.lazy()
lf.collect()

ts,value,scada_id,tank_name,uom
datetime[μs],f64,str,str,str
2019-04-05 01:41:07.809116,76.456344,"""98840W2""","""WaterTank2Level""","""in"""
2019-04-05 00:41:07.809116,76.349873,"""98840W2""","""WaterTank2Level""","""in"""
2019-04-04 23:41:07.809116,76.283003,"""98840W2""","""WaterTank2Level""","""in"""
2019-04-04 22:41:07.809116,76.287785,"""98840W2""","""WaterTank2Level""","""in"""
2019-04-04 21:41:07.809116,76.388698,"""98840W2""","""WaterTank2Level""","""in"""
…,…,…,…,…
2023-08-15 00:52:13.787936,365.600882,"""98840W2""","""WaterTank2Volume""","""bbl"""
2023-08-14 23:52:13.787936,363.953501,"""98840W2""","""WaterTank2Volume""","""bbl"""
2023-08-14 22:52:13.787936,362.274663,"""98840W2""","""WaterTank2Volume""","""bbl"""


In [6]:
tank_metrics = ["Level", "Volume", "InchesUntilAlarm", "InchesToESD"]
tank_types = ["Water", "Oil"]

tank_metrics_str= "|".join(tank_metrics)
tank_types_str = "|".join(tank_types)

pattern = f'^(?<is_esd>ESD-)?(?<tank_type>{tank_types_str})Tank(?<tank_number>[0-9]*)(?<tank_metric>{tank_metrics_str})'
lf = lf.with_columns(separated_metrics = pl.col("tank_name").str.extract_groups(pattern))
lf = lf.unnest("separated_metrics")
lf.collect()

ts,value,scada_id,tank_name,uom,is_esd,tank_type,tank_number,tank_metric
datetime[μs],f64,str,str,str,str,str,str,str
2019-04-05 01:41:07.809116,76.456344,"""98840W2""","""WaterTank2Level""","""in""",null,"""Water""","""2""","""Level"""
2019-04-05 00:41:07.809116,76.349873,"""98840W2""","""WaterTank2Level""","""in""",null,"""Water""","""2""","""Level"""
2019-04-04 23:41:07.809116,76.283003,"""98840W2""","""WaterTank2Level""","""in""",null,"""Water""","""2""","""Level"""
2019-04-04 22:41:07.809116,76.287785,"""98840W2""","""WaterTank2Level""","""in""",null,"""Water""","""2""","""Level"""
2019-04-04 21:41:07.809116,76.388698,"""98840W2""","""WaterTank2Level""","""in""",null,"""Water""","""2""","""Level"""
…,…,…,…,…,…,…,…,…
2023-08-15 00:52:13.787936,365.600882,"""98840W2""","""WaterTank2Volume""","""bbl""",null,"""Water""","""2""","""Volume"""
2023-08-14 23:52:13.787936,363.953501,"""98840W2""","""WaterTank2Volume""","""bbl""",null,"""Water""","""2""","""Volume"""
2023-08-14 22:52:13.787936,362.274663,"""98840W2""","""WaterTank2Volume""","""bbl""",null,"""Water""","""2""","""Volume"""


In [7]:
lf = lf.drop("tank_name", "is_esd", "tank_type", "tank_number")
lf.collect()

ts,value,scada_id,uom,tank_metric
datetime[μs],f64,str,str,str
2019-04-05 01:41:07.809116,76.456344,"""98840W2""","""in""","""Level"""
2019-04-05 00:41:07.809116,76.349873,"""98840W2""","""in""","""Level"""
2019-04-04 23:41:07.809116,76.283003,"""98840W2""","""in""","""Level"""
2019-04-04 22:41:07.809116,76.287785,"""98840W2""","""in""","""Level"""
2019-04-04 21:41:07.809116,76.388698,"""98840W2""","""in""","""Level"""
…,…,…,…,…
2023-08-15 00:52:13.787936,365.600882,"""98840W2""","""bbl""","""Volume"""
2023-08-14 23:52:13.787936,363.953501,"""98840W2""","""bbl""","""Volume"""
2023-08-14 22:52:13.787936,362.274663,"""98840W2""","""bbl""","""Volume"""


In [8]:
aggregated_lf = lf.group_by(["tank_metric", "uom"]).agg([
    pl.col("ts").alias("timestamps"),
    pl.col("value").alias("values")
])

aggregated_lf.collect()

tank_metric,uom,timestamps,values
str,str,list[datetime[μs]],list[f64]
"""Level""","""in""","[2019-04-05 01:41:07.809116, 2019-04-05 00:41:07.809116, … 2019-04-03 02:41:07.809116]","[76.456344, 76.349873, … 78.172158]"
"""Volume""","""bbl""","[2023-08-16 19:52:13.787936, 2023-08-16 18:52:13.787936, … 2023-08-14 20:52:13.787936]","[357.148125, 357.041653, … 358.863939]"
